# Amicable numbers

[problem 21](https://projecteuler.net/problem=21)
> Let $d(n)$ be defined as the sum of proper divisors of $n$ (numbers less than n which divide evenly into $n$).
If $d(a) = b$ and $d(b) = a$, where $a \neq b$, then a and b are an amicable pair and each of a and b are called amicable numbers.
>
> For example, the proper divisors of $220$ are $1, 2, 4, 5, 10, 11, 20, 22, 44, 55, 110$ therefore $d(220) = 284$ 
> The proper divisors of $284$ are $1, 2, 4, 71, 142$ so $d(284) = 220$
>
>Evaluate the sum of all the amicable numbers under $10000$.

In [1]:
function euler21PrimeFactorization() {
    
    var dCache = Object.create(null);
    var primes;
    
    function amicable(n) {
        primes = eratosthenes(n);    
        var sum = 0;
        for (var i = 220; i < n; i++) {
            if (d(i) != i && d(d(i)) == i) {
                sum += i;
            }
        }
        return sum;
    }
    
    function eratosthenes(n) {
        var prms = [];
        var limit = n-1;
        if (limit > 2) {
            var sqrtlmt = (Math.sqrt(limit) - 3) >> 1;
            var lmt = (limit - 3) >> 1;
            var bfsz = (lmt >> 5) + 1
            var buf = [];
            for (var i = 0; i < bfsz; i++)
                buf.push(0);
            for (var i = 0; i <= sqrtlmt; i++)
                if ((buf[i >> 5] & (1 << (i & 31))) == 0) {
                    var p = i + i + 3;
                    for (var j = (p * p - 3) >> 1; j <= lmt; j += p)
                        buf[j >> 5] |= 1 << (j & 31);
                }
            prms.push(2)
            for (var i = 0; i <= lmt; i++)
                if ((buf[i >> 5] & (1 << (i & 31))) == 0)
                    prms.push(i + i + 3);
        }
        return prms;
    }
    
    function primeFactors(n) {
        var factors = Object.create(null);
        for (var i = 0, p = primes[0]; p*p <= n; p=primes[++i]) {
            var power = 0;
            while (!(n % p)) {
                power++;
                n /= p;
            }
            if (power > 0) factors[p] = power;
        }
        if (n !== 1) factors[n] = 1;
        return factors;
    }
    
    function d(n) {
        if (!(n in dCache)) {
            var factors = primeFactors(n);
            dCache[n] = -n + Object.keys(factors)
                .map(function(key) {
                    var sum = 0;
                    var prime = +key;
                    for (var i = 0; i <= factors[key]; i++) {
                        sum += Math.pow(prime, i);
                    }
                    return sum;
                }).reduce(function(a, b) {
                    return a*b;
                }, 1);
        }
        return dCache[n];
    }
    
    return amicable(10000);
}

euler21PrimeFactorization();

31626

In [2]:
function euler21SumDivisorSieve() {
    
    function sumDivisors(n) {
        var sumDivs = Array.apply(null, Array(n)).map(function() {
            return 0;
        });
        for (var i = 1; i < n/2; i++) {
            for (var j = i+i; j < n; j+=i) {
                sumDivs[j] += i;
            }
        }
        return sumDivs;
    }
    
    var sumDivs = sumDivisors(25321);
    var sum = 0;
    for (var i = 220; i < 10000; i++) {
        if (sumDivs[i] != i && sumDivs[sumDivs[i]] == i) {
            sum += i;
        }
    }
    return sum;
}

euler21SumDivisorSieve();

31626

## Benchmarks

In [3]:
$$async$$ = true;
var Benchmark = require('benchmark');
var unit = [' s', ' ms', ' microseconds', ' ns', ' ps']
var suite = new Benchmark.Suite;
suite.add('euler21#PrimeFactorization', euler21PrimeFactorization);
suite.add('euler21#SumDivisorSieve', euler21SumDivisorSieve);
suite.on('complete', function() {
    var result = {};
    this.forEach(function(r) {
        var p = Math.ceil((Math.log(r.hz) / Math.LN10) / 3);
        result[r.name] = 1 / r.hz * Math.pow(10, p*3) + unit[p];
    });
    $$done$$(result);
});
suite.run({'async':true});

{ 'euler21#PrimeFactorization': '55.5312495882353 ms',
  'euler21#SumDivisorSieve': '4.918014784922394 ms' }